In [ ]:
import pandas as pd, numpy as np
import ast

In [ ]:
dataframe = pd.read_csv("../scraping/out.csv")

In [ ]:
dataframe.columns

In [ ]:
dataframe.shape

In [ ]:
dataframe.tail()

In [ ]:
dataframe.dropna(subset=['title','singers','download_link', 'year'], inplace= True)
dataframe.reset_index(inplace=True)

In [ ]:
#checking for null values
null_lyricist = dataframe.isna()['lyricist'].sum()
null_directors = dataframe.isna()['directors'].sum()
null_posters = dataframe.isna()['poster'].sum()
print(null_directors, null_lyricist, null_posters)

In [ ]:
def convert_to_words(words):
    try:
        if not words:
            return []
        list = ast.literal_eval(words)
        return " " + " ".join([one_word(s) for s in list])
    except:
        return " "
def one_word(s):
    return "".join(s.lower().split())

In [ ]:
new_df = pd.DataFrame()
new_df['tags'] = dataframe['title']
new_df['tags'] += dataframe['singers'].apply(convert_to_words)
new_df['tags'] += dataframe['directors'].apply(convert_to_words)
new_df['tags'] += dataframe['lyricist'].apply(convert_to_words)
new_df['tags'] += dataframe['genre'].apply(convert_to_words)
new_df['tags'] += " " + dataframe['album']

In [ ]:
new_df.head().iloc[0].tags

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
type(new_df['tags'])

In [ ]:
vector.shape

In [194]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [208]:
similarity.shape

(923, 923)

In [209]:
sorted_indices = np.array([np.argsort(-row)[1: 51] for row in similarity])
np.save('similarity.npy', sorted_indices)

In [ ]:
def recommend(song):
    index = dataframe[dataframe['title'] == song].index[0]
    similar = enumerate(similarity[index])
    r_songs = sorted(similar, key = lambda x : -x[1])[1:51]
    list_of_songs = []
    for sng in r_songs:
        list_of_songs.append(dataframe.iloc[sng[0]].to_dict())
    return list_of_songs

In [ ]:
r = recommend(dataframe.iloc[0].title)

In [ ]:
[i['index'] for i in r][:50]

In [ ]:
#for saving the database
save_df = pd.DataFrame()
save_df = dataframe[['title', 'download_link', 'poster', 'year']]

In [ ]:
save_df.head()

In [ ]:
save_df.to_pickle('./songs.pkl')